In [131]:
# Installing the latest version Ollama
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10941    0 10941    0     0  34158      0 --:--:-- --:--:-- --:--:-- 34190
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [132]:
# Setting up the model, Mistral is the LLM used.
OLLAMA_MODEL='mistral:latest'

# Adding the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL

import subprocess
import time

# Starting ollama on the server
command = "nohup ollama serve&"

process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid)
time.sleep(5)

!ollama -v

mistral:latest
Process ID: 27029
ollama version is 0.1.41


In [133]:
!pip install llama-index-embeddings-huggingface --quiet
!pip install llama-index-llms-ollama --quiet
!pip install llama-index-vector-stores-chroma --quiet
!pip install llama-index llama-index-experimental --quiet
!pip install llama-index ipywidgets --quiet
!pip install llama-index-llms-huggingface --quiet
!pip install chromadb --quiet

# Importing necessary modules from the llama_index library and Chroma_db
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever
from llama_index.core import get_response_synthesizer, PromptTemplate, ChatPromptTemplate
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.schema import Node, NodeWithScore

In [134]:
# Querying the model via the command line. First time running it will "pull" (import) the model
!ollama run $OLLAMA_MODEL "Give me short summary of Warwick Business School."

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙  Warwick Business School (WBS) is a prominent business school located at the University of Warwick in Coventry, UK. It was established in 1967 and is one of the first business schools to be established in Britain. The school offers undergraduate, postgraduate, executive education, and doctoral programs across various disciplines such as finance, management, marketing, economics, entrepreneurship, and more.

WBS is known for its research excellence, with a strong focus on understanding how organizations work, and how management ideas can be developed to improve organizational effectiveness. It has consistently ranked among the top business schools globally by publications like the Financial Times and The Economist. The school also places great emphasis on corporate connections, offering numerous opportunities for students to network with industry professionals and gain practical experience.



In [135]:
llm = Ollama(model=OLLAMA_MODEL, request_timeout=240.0)

# Launching a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specifying the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [136]:
# Pre-processing of the data to make LLM understand better. Write each row to a txt file and add info about each row.
def text_create(row):
  import datetime

  dtobj = datetime.datetime.strptime(str(row[2])[4:6], "%m")
  month = dtobj.strftime("%B")

  text = f'''Sign: {row[0]},
             Day: {str(row[2])[6:8]},
             Month: {month},
             Horoscope:
             {row[3]}'''

  name = f"{row[0]}_{month}_{str(row[2])[6:8]}"

  with open(f'/content/data/{name}.txt', 'w+') as textfile:
      textfile.write(text)

In [137]:
import pandas as pd

dataset = pd.read_csv('horoscopes.csv')
dataset = dataset[dataset["category"] == "general"]

!mkdir -p '/content/data/'


for row in dataset.values:
  text_create(row=row)


# Loading document
reader = SimpleDirectoryReader('/content/data/')
docs = reader.load_data()

In [138]:
# Creating client 'db' and a database 'chroma_db'
db = chromadb.PersistentClient(path="./chroma_db")

# Creating a collection named RAG in the db
chroma_collection = db.get_or_create_collection("RAG")

# Setting up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Creating the index
index = VectorStoreIndex.from_documents(
    docs,
    storage_context = storage_context,
    embed_model = embed_model
)

print(chroma_collection)
print(f'Collection name is: {chroma_collection.name}')

name='RAG' id=UUID('2c5c6b42-ddcf-4568-97de-61698ba7c9a9') metadata=None tenant='default_tenant' database='default_database'
Collection name is: RAG


In [139]:
# Generating a customized query engine including Chat memory, retriever, post-processor, response synthesizer and prompt template,

# Chat Memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Retriever
retriever = VectorIndexRetriever(
    index=index,
    streaming=True,
    similarity_top_k=12,
)

# Post-Processor
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.6)

EmptyNode = [NodeWithScore(node=Node(text=""), score=0.0)]

# Response Synthesizer
synthesizer = get_response_synthesizer(
      streaming=True, response_mode="compact",
)

# Prompt Template
qa_prompt_str = (
    "You are able to have normal interactions, as well as talk"
    "about the horoscope forecasts.\n"
    "---------------------\n"
    "Chat history is provided below:\n"
    "{chat_history}\n"
    "---------------------\n"
    "Forecasts about relevant horoscopes are given in the context below:\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the Horoscope information from the context with chat history and not prior knowledge, "
    "answer the question: {query_str}\n"
)


chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "If no context is provided, answer from chat history. Even if chat history does not provide any relevant information, still answer the question."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

qa_prompt = ChatPromptTemplate(chat_text_qa_msgs)

# Customized Query Engine and assembly of all components in the engine

class RAGStringQueryEngine(CustomQueryEngine):

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: llm
    qa_prompt: ChatPromptTemplate
    pipeline_memory: ChatMemoryBuffer
    postprocessor: SimilarityPostprocessor

    def custom_query(self, query_str: str):

        chat_history_obj = pipeline_memory.get()
        chat_history = "\n".join([str(x) for x in chat_history_obj])

        user_msg = ChatMessage(role="user", content=query_str)
        pipeline_memory.put(user_msg)

        nodes = self.retriever.retrieve(query_str)
        nodes_p = postprocessor.postprocess_nodes(nodes)
        context_str = "\n\n".join([n.node.get_content() for n in nodes_p])

        if context_str == "":
          nodes_p = EmptyNode

        question_with_context = qa_prompt.format(chat_history=chat_history, context_str=context_str, query_str=query_str)
        response_obj = self.response_synthesizer.synthesize(question_with_context, nodes_p)

        llm_msg = ChatMessage(role="assistant", content=str(response_obj))
        pipeline_memory.put(llm_msg)

        return response_obj

In [140]:
# Function to deal with Multi-Hop Reasoning and make the chat memory save all the subsequentially dependent responses

def multi_hop(msgs, metadata=False):

  # Initializing memory
  pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

  # Calling the customized query engine
  query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
    pipeline_memory=pipeline_memory,
    postprocessor=postprocessor
  )

  # Displaying the responses
  for msg in msgs:

    print("-"*30+"\n")
    print(f"User:\n {msg}\n")
    print("Assistant:\n")

    res = query_engine.query(msg)
    res.print_response_stream()
    print("\n")
    print("-"*30)

    if metadata == True:
      print("Metadata:")
      print(res.metadata)

In [141]:
# Custom Query Engine
query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
    pipeline_memory=pipeline_memory,
    postprocessor=postprocessor
  )


In [94]:
# Initializing memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Test question 1
response1 = query_engine.query("My sign is Scorpio. What is the horoscope of 23 September?")
response1.print_response_stream()

 A reassessment of your financial goals may be needed now, as some of your expectations may be unrealistic and you might need to look at them in a more practical light. This doesn't mean they won't happen, just that they're apt to take a little longer than you'd like. Think of it as an opportunity to develop patience. Rework your budget and hang in there!


In [95]:
# Metadata of question 1
response1.metadata

{'021df406-3863-4322-84f4-fdd1187f2a17': {'file_path': '/content/data/scorpio_September_23.txt',
  'file_name': 'scorpio_September_23.txt',
  'file_type': 'text/plain',
  'file_size': 468,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '77960a11-a882-4567-9585-a28bab29c33a': {'file_path': '/content/data/scorpio_August_23.txt',
  'file_name': 'scorpio_August_23.txt',
  'file_type': 'text/plain',
  'file_size': 484,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '8fb4a306-202b-406b-acfe-17781e544819': {'file_path': '/content/data/virgo_September_23.txt',
  'file_name': 'virgo_September_23.txt',
  'file_type': 'text/plain',
  'file_size': 486,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '6be191bb-4d9e-490c-ae47-143fd3c230fe': {'file_path': '/content/data/scorpio_December_23.txt',
  'file_name': 'scorpio_December_23.txt',
  'file_type': 'text/plain',
  'file_size': 483,
  'creation_date': '2024-06-05',
  'last

In [96]:
# Initializing memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Test question 2
response2 = query_engine.query("Write me the horoscope of the sign Pisces for May 18.")
response2.print_response_stream()

 The horoscope for the sign Pisces on May 18 suggests that some differences of opinion concerning your love life may come up today. You could find your beloved taking more liberties than you allow yourself. Instead of resenting it, follow your partner's lead. This could be a chance to realize that you need more novelty in the emotional realm. Be open to looking into the possibilities. The day ahead could bring you just what you seek.


In [97]:
# Metadata of question 2
response2.metadata

{'298ce16b-31db-482c-a0a2-0d9ddbbf0a6f': {'file_path': '/content/data/pisces_May_18.txt',
  'file_name': 'pisces_May_18.txt',
  'file_type': 'text/plain',
  'file_size': 484,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'bf1df7bf-dccc-42d4-bec0-217d86daf915': {'file_path': '/content/data/pisces_May_11.txt',
  'file_name': 'pisces_May_11.txt',
  'file_type': 'text/plain',
  'file_size': 430,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'b9d53ac7-b33a-41a4-81b7-a5c69f938d59': {'file_path': '/content/data/aries_May_18.txt',
  'file_name': 'aries_May_18.txt',
  'file_type': 'text/plain',
  'file_size': 440,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'b6e0dae5-241b-4d0a-a41d-cd56d406af53': {'file_path': '/content/data/pisces_June_18.txt',
  'file_name': 'pisces_June_18.txt',
  'file_type': 'text/plain',
  'file_size': 489,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '9bf2360b-4

In [99]:
# Initializing memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Test question 3
response3 = query_engine.query("What are the horoscopes of all signs in 7 August?")
response3.print_response_stream()

 Libra: Meditation, dreams, or visions could bring amazing insights your way. You might find them too bizarre to believe, at least at first, but follow-up research could reveal that what you've come up with is quite credible. Keep a journal of these revelations because you might remember the gist but forget the details. They could be useful later when you pass your ideas on to others.

Scorpio: You're an adventurous soul, Scorpio. Are you thinking about a little mischief today? Are you going to propose that you and a friend or perhaps your partner skip town for a day and forget about work, chores, and other routine matters if possible? You've been bored lately, so who could blame you? If you decide to do this, make sure everyone knows what's up. You don't want to burn any bridges.

Gemini: Some unexpected but very welcome visitors could arrive today, Gemini. They might bring some people you don't know but should connect with right away. Expect interesting news, fascinating information,

In [100]:
# Metadata of question 3
response3.metadata

{'7e26ad1c-b81f-441f-a7df-284c65f0cfe2': {'file_path': '/content/data/libra_August_07.txt',
  'file_name': 'libra_August_07.txt',
  'file_type': 'text/plain',
  'file_size': 486,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'a548ef11-a853-48d7-80f2-e57047eac8c6': {'file_path': '/content/data/scorpio_August_07.txt',
  'file_name': 'scorpio_August_07.txt',
  'file_type': 'text/plain',
  'file_size': 495,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '36a84d21-3bb7-43ca-9602-1d809e739cb4': {'file_path': '/content/data/gemini_August_07.txt',
  'file_name': 'gemini_August_07.txt',
  'file_type': 'text/plain',
  'file_size': 503,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'f9fcee4f-2f03-47db-88e3-aed6996702a4': {'file_path': '/content/data/pisces_August_07.txt',
  'file_name': 'pisces_August_07.txt',
  'file_type': 'text/plain',
  'file_size': 476,
  'creation_date': '2024-06-05',
  'last_modified_date': '20

In [101]:
# Initializing memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Test question 4
response4 = query_engine.query("What is the general horoscope for Gemini?")
response4.print_response_stream()

 The horoscope for Gemini, based on the provided examples, seems to suggest a mix of various aspects. Some days may bring clarity and understanding, allowing for self-expression and connection with others (March 27). On other days, Gemini might find themselves driven by desire, needing to show their determination to achieve what they want (July 3). The horoscope also highlights the importance of clear communication in relationships, with an emphasis on understanding and strengthening bonds (January 6). Additionally, there are days where practicality is urged over dreaminess (July 30), and days when Gemini should focus on personal growth and self-care (November 3). Overall, the horoscope for Gemini seems to be about balance between self-expression, determination, communication, and personal development.


In [102]:
# Metadata of question 4
response4.metadata

{'ad55f98b-15c8-4894-a2bd-782f05f4701d': {'file_path': '/content/data/gemini_January_19.txt',
  'file_name': 'gemini_January_19.txt',
  'file_type': 'text/plain',
  'file_size': 452,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '7ab9a8ed-181f-481c-9509-a3ad7894a2ad': {'file_path': '/content/data/gemini_March_27.txt',
  'file_name': 'gemini_March_27.txt',
  'file_type': 'text/plain',
  'file_size': 443,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'dcc307fd-b100-4587-9fdc-4d42615015e5': {'file_path': '/content/data/gemini_July_03.txt',
  'file_name': 'gemini_July_03.txt',
  'file_type': 'text/plain',
  'file_size': 456,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '2da97400-69bf-4726-9dab-39bcd80030e6': {'file_path': '/content/data/gemini_July_05.txt',
  'file_name': 'gemini_July_05.txt',
  'file_type': 'text/plain',
  'file_size': 478,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05

In [103]:
# Initializing memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Test question 5. Just wording is different from test question 4
response5 = query_engine.query("Summarize the horoscope for Gemini.")
response5.print_response_stream()

 The horoscope for Gemini suggests a series of days that encourage self-discovery, intellectual curiosity, creativity, and personal growth. One day might stimulate an interest in a particular field or topic, prompting you to learn more about it. Another day encourages introspection and questioning one's beliefs, with suggestions for yoga, meditation, and dream journaling. Creative projects, particularly those involving music, painting, writing, or business partnerships, are highlighted on specific days. The horoscope also emphasizes the importance of joining groups that share your interests and pursuing subjects you might otherwise deem "non-serious." There is a focus on personal relationships, with an emphasis on taking advantage of the energy from today's aspects to strengthen romantic involvements or seek deeper commitments. Additionally, there are indications that you may find wisdom in others and that you should trust your intuition for self-discovery.


In [104]:
# Metadata of question 5
response5.metadata

{'9dda9a1c-d2a2-411f-aa67-6ba03f0c97aa': {'file_path': '/content/data/gemini_October_11.txt',
  'file_name': 'gemini_October_11.txt',
  'file_type': 'text/plain',
  'file_size': 480,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '4f9cdfbb-34c0-4420-986f-a2395da33881': {'file_path': '/content/data/gemini_April_12.txt',
  'file_name': 'gemini_April_12.txt',
  'file_type': 'text/plain',
  'file_size': 493,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'a1f14f2c-0cf2-4a7c-8012-a47b914b4a38': {'file_path': '/content/data/gemini_May_31.txt',
  'file_name': 'gemini_May_31.txt',
  'file_type': 'text/plain',
  'file_size': 478,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '53a23600-f7e0-4cad-8069-f38957ef6ced': {'file_path': '/content/data/gemini_April_27.txt',
  'file_name': 'gemini_April_27.txt',
  'file_type': 'text/plain',
  'file_size': 488,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05

In [123]:
# Initializing memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Test question 6
response6 = query_engine.query("Tell how compatible Scorpio and Aries are.")
response6.print_response_stream()

 According to astrology, the compatibility between a Scorpio and an Aries can be a bit challenging due to their differences in personality traits and behaviors. Scorpios tend to be intense, passionate, and secretive, while Aries are assertive, energetic, and adventurous. However, when they find common ground, such as mutual respect and understanding, they can form a strong bond that helps each other grow.


In [125]:
# Metadata of question 6
response6.metadata

{'24c4e2fd-2a22-4e24-9cca-3e4fb2e4b2ce': {}}

In [108]:
# Initializing memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Test question 7
response7 = query_engine.query("Explain the compatibility of Virgo and Leo for 10 October.")
response7.print_response_stream()

 On October 10th, both Virgo and Leo are provided horoscopes in the context. For Virgo, it is stated that their sweetheart may seem upset and unlikely to want to talk about it. This could potentially create some tension or frustration, especially given that Virgo generally dislikes being kept in the dark.

On the other hand, for Leo, the horoscope mentions emotional discussions with their partner could move them to tears, indicating a potentially sensitive or emotional state. They might have personal and financial issues to resolve on this day.

Taking into account both signs' horoscopes, it seems that on October 10th, there might be some compatibility challenges for Virgo and Leo due to the potential tension and emotional turmoil each may experience individually. However, understanding and empathy could help them navigate through these challenges.


In [109]:
# Metadata of question 7
response7.metadata

{'4f47ab24-2569-4cc7-9a1f-ee39b58d3440': {'file_path': '/content/data/virgo_October_23.txt',
  'file_name': 'virgo_October_23.txt',
  'file_type': 'text/plain',
  'file_size': 447,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 '77477710-139d-42f4-82f0-288317af98c3': {'file_path': '/content/data/virgo_October_10.txt',
  'file_name': 'virgo_October_10.txt',
  'file_type': 'text/plain',
  'file_size': 484,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'aeda3b2f-4cab-4450-82e5-8243cac23d42': {'file_path': '/content/data/leo_October_29.txt',
  'file_name': 'leo_October_29.txt',
  'file_type': 'text/plain',
  'file_size': 484,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-06-05'},
 'e3907b96-35bd-41f8-88b6-9de299812a03': {'file_path': '/content/data/virgo_October_16.txt',
  'file_name': 'virgo_October_16.txt',
  'file_type': 'text/plain',
  'file_size': 464,
  'creation_date': '2024-06-05',
  'last_modified_date': '2024-0

In [142]:
 # Multi-Hop Reasoning System test questions
mhrs = [
    "My sign is Capricorn. For September 19, write me a horoscope.",
    "Which date is mentioned in the chat history?",
    "My ascendant sign is Cancer. For the date mentioned in the chat history, tell me the horoscope for Cancer.",
    "Summarize the chat history so far.",
]

# Initializing memory
pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)
multi_hop(mhrs, metadata=True)

------------------------------

User:
 My sign is Capricorn. For September 19, write me a horoscope.

Assistant:

 On September 19th, as a Capricorn, you are advised to initiate a major change in your life. This day encourages you to break free from the ordinary and venture into something exciting. Consider enrolling in an online class that expands your mind, such as yoga, tarot, or martial arts. You have a great deal of energy today which will help you maintain confidence and endurance as you work on establishing this new life-enhancing endeavor.


------------------------------
Metadata:
{'ebc7d4ad-6e60-4b3f-9874-f4d6dc730d95': {'file_path': '/content/data/capricorn_September_19.txt', 'file_name': 'capricorn_September_19.txt', 'file_type': 'text/plain', 'file_size': 494, 'creation_date': '2024-06-05', 'last_modified_date': '2024-06-05'}, 'cde8235f-126b-45ef-a3ce-5104404a46b2': {'file_path': '/content/data/capricorn_September_09.txt', 'file_name': 'capricorn_September_09.txt', 'file_t